## combining individual coverage data into a mean-normalized coverage

## Plan:

+ for the PanelofNormals, create a coverage file for each PON bam and each chromosome
+ Pon bams should optimally not contain CNV or there should be a lot of them to reduce the std
+ put all the files into a big matrix and normalize coverages and produce an average coverage (+ std) for the exonic space
+ compare the tumor samples against that PONcoverage to get differences in CNV

### have to look at the Kenichi-CNV-graphs to identify the samples from FDAH1 without CNVs in the normals
+ [1, 2, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

### alternative approach
+ get all samples and remove the outliers from std

In [80]:
# set the paths
home = '/Users/mahtin'
home = '/Users/martinscience'
testdata = f"{home}/Dropbox/Icke/Work/somVar/testdata"
tooldata = f"{home}/Dropbox/Icke/Work/somVar/tooldata"
shell_path = "../shell"
static_path = f"{home}/Dropbox/Icke/Work/static"
bed_path = f"{static_path}/bed_files/SureSelect/hg38"

cnvdata = f"{tooldata}/myCNVdata/"
bedCov_path = f"{cnvdata}/bedCov"
bedCov_path = f"{home}/mount/scratch/develop/PONcoverage/bedCov"
output_path = f"{cnvdata}/output/PONcoverage"

In [81]:
bedCov_path

'/Users/martinscience/mount/scratch/develop/PONcoverage/bedCov'

In [3]:
test = pd.read_csv(f'{bedCov_path}/009_B.chr5.bedCov', sep='\t', compression="gzip")
test[:13]

,Chr,Pos,Coverage,ExonPos
0,chr5,140300,184.30,33
1,chr5,140350,242.62,83
2,chr5,140400,288.67,133
3,chr5,140450,347.07,183
4,chr5,140500,406.70,233
5,chr5,140550,480.91,283
6,chr5,140600,544.79,333
7,chr5,140650,523.75,383
8,chr5,140700,384.47,433
9,chr5,140750,187.23,483


### get all the normal samples from the PON list into df for normalization and averaging

In [61]:
# try with pd merge
import os
def combine_coverage(PONcov_path, chrom, sample_list):
    cov_df = pd.DataFrame(columns=['ExonPos', 'Pos'])
    for sample in sample_list:
        file = os.path.join(PONcov_path, f"{sample}.{chrom}.bedCov")
        if not os.path.isfile(file):
            continue
        print(f"Reading {sample} from {file}.")
        df = pd.read_csv(file, sep='\t', compression='gzip').loc[:,['Pos', 'ExonPos', 'Coverage']].rename(columns={'Coverage':sample})
        cov_df = cov_df.merge(df, on=['ExonPos', 'Pos'], how='outer')
    cov_df = cov_df.fillna(0).sort_values('ExonPos')
    cov_df['Chr'] = chrom
    # reorder columns
    cols = ['Chr', 'Pos', 'ExonPos'] + list(cov_df.columns)[2:-1]
    return cov_df.loc[:,cols]

In [62]:
sample_list = [f"{str(s).zfill(3)}_B" for s in range(45)]
cov_df = combine_coverage(bedCov_path, "chr7", sample_list)
cov_df

Reading 001_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/001_B.chr7.bedCov.
Reading 002_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/002_B.chr7.bedCov.
Reading 003_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/003_B.chr7.bedCov.
Reading 004_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/004_B.chr7.bedCov.
Reading 005_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/005_B.chr7.bedCov.
Reading 006_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/006_B.chr7.bedCov.
Reading 008_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/008_B.chr7.bedCov.
Reading 009_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/009_B.chr7.bedCov.
Reading 010_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/010_B.chr7.bedCov.
Reading 011_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/011_B.chr7.bedCov.


,Chr,Pos,ExonPos,001_B,002_B,003_B,004_B,005_B,006_B,008_B,...,029_B,031_B,033_B,034_B,036_B,039_B,040_B,041_B,042_B,044_B
0,chr7,35350,967,19.60,11.43,20.05,73.63,4.03,4.76,34.20,...,10.24,11.39,0.00,6.23,2.81,6.20,7.00,13.67,2.50,1.47
1,chr7,35400,1017,28.99,9.91,34.33,97.54,3.43,5.52,55.41,...,15.01,19.55,0.00,5.59,2.43,13.26,14.71,24.39,2.38,1.34
2,chr7,35450,1067,30.32,4.63,36.78,94.73,1.45,3.81,63.48,...,17.81,22.45,0.00,2.90,1.16,16.13,16.55,26.04,1.88,0.87
3,chr7,36200,1112,51.93,0.49,0.46,0.64,0.00,56.72,1.97,...,0.00,8.13,0.00,7.85,11.52,0.00,0.00,0.00,0.00,0.00
4,chr7,36250,1162,53.62,2.78,0.96,1.64,0.00,49.94,2.97,...,0.00,5.95,0.00,8.63,9.85,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46362,chr7,159144650,2423334,475.69,609.91,607.29,586.38,333.41,170.54,367.27,...,61.29,74.82,137.21,121.58,102.12,119.29,77.48,70.24,67.55,83.36
46363,chr7,159144700,2423384,384.81,476.35,480.09,483.67,272.19,156.43,289.89,...,43.20,57.93,115.75,90.37,82.08,105.85,66.64,59.81,54.97,65.07
46364,chr7,159144750,2423434,306.03,385.24,390.18,409.29,216.75,138.26,237.72,...,37.12,45.26,105.61,71.26,74.28,101.79,53.69,52.63,47.55,54.10
46365,chr7,159144800,2423484,228.64,300.84,321.24,316.32,171.66,107.90,196.13,...,33.18,29.96,82.46,58.28,58.47,92.03,38.82,44.61,36.21,37.59


### make a stacked "tidy version" of the coverage df for vizualisation in tidyverse

In [18]:
def tidy_df(df):
    return df.drop(columns=['Chr', 'Pos']).set_index('ExonPos').stack().reset_index().rename(columns={'level_1':'sample', 0:'Coverage'})

cov_df.to_csv(f"{output_path}/PON_coverage.csv", sep='\t', index=False)

tidy_df(cov_df).to_csv(f"{output_path}/PON_coverage_tidy.csv", sep='\t', index=False)

# actually I can tidy the data in R with gather:

### normalize the coverage to coverage 100

In [63]:
def normalize_coverage(cov_df, norm_cov=100):
    norm_df = cov_df.set_index(['Chr','Pos','ExonPos'])
    norm_df = norm_df / norm_df.mean() * norm_cov
    return norm_df.reset_index()

In [20]:
norm_df = normalize_coverage(cov_df)
norm_df

,Chr,Pos,ExonPos,001_B,002_B,003_B,004_B,005_B,006_B,008_B,...,029_B,031_B,033_B,034_B,036_B,039_B,040_B,041_B,042_B,044_B
0,chr7,35350,967,4.033958,2.313776,3.922525,12.226485,0.703237,0.822889,7.261974,...,5.599328,7.034867,0.000000,3.480870,1.543389,3.009409,3.616486,6.796135,1.246768,0.697537
1,chr7,35400,1017,5.966553,2.006082,6.716224,16.196813,0.598537,0.954275,11.765672,...,8.207608,12.074771,0.000000,3.123285,1.334675,6.436252,7.599787,12.125656,1.186923,0.635850
2,chr7,35450,1067,6.240286,0.937251,7.195536,15.730204,0.253026,0.658657,13.479243,...,9.738675,13.865914,0.000000,1.620309,0.637129,7.829317,8.550406,12.945965,0.937569,0.412828
3,chr7,36200,1112,10.687931,0.099191,0.089993,0.106274,0.000000,9.805522,0.418307,...,0.000000,5.021375,0.000000,4.386008,6.327347,0.000000,0.000000,0.000000,0.000000,0.000000
4,chr7,36250,1162,11.035757,0.562756,0.187812,0.272327,0.000000,8.633423,0.630645,...,0.000000,3.674930,0.000000,4.821815,5.410101,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47121,chr7,159144650,2423334,97.903753,123.464123,118.808504,97.370181,58.180198,29.482259,77.985534,...,33.513945,46.211477,63.701295,67.930045,56.089291,57.901994,40.029333,34.920299,33.687667,39.555581
47122,chr7,159144700,2423384,79.199359,96.427563,93.923455,80.314873,47.497280,27.042979,61.554787,...,23.622164,35.779616,53.738247,50.492171,45.082345,51.378373,34.428946,29.734953,27.413931,30.876699
47123,chr7,159144750,2423434,62.985317,77.984159,76.333716,67.963848,37.822975,23.901824,50.477091,...,20.297563,27.954176,49.030637,39.814896,40.798204,49.407696,27.738447,26.165367,23.713525,25.671268
47124,chr7,159144800,2423484,47.057357,60.899062,62.846488,52.525897,29.954749,18.653311,41.645936,...,18.143134,18.504355,38.282988,32.562618,32.114580,44.670304,20.055998,22.178168,18.058186,17.837024


In [22]:
norm_df.to_csv(f"{output_path}/PON_coverage_norm.csv", sep='\t', index=False)
tidy_df(norm_df).to_csv(f"{output_path}/PON_coverage_normtidy.csv", sep='\t', index=False)

### compute the mean of all the coverages

In [23]:
def add_mean(norm_df):
    norm_df = norm_df.set_index(['Chr', 'Pos', 'ExonPos'])
    norm_df['meanCov'] = norm_df.mean(axis=1)
    norm_df['medianCov'] = norm_df.median(axis=1)
    norm_df['std'] = norm_df.std(axis=1)
    return norm_df.reset_index()
mean_df = add_mean(norm_df)
mean_df.to_csv(f"{output_path}/PON_coverage_mean.csv", sep='\t', index=False)
tidy_df(mean_df).to_csv(f"{output_path}/PON_coverage_mean.tidy.csv", sep='\t', index=False)
mean_df

,Chr,Pos,ExonPos,001_B,002_B,003_B,004_B,005_B,006_B,008_B,...,034_B,036_B,039_B,040_B,041_B,042_B,044_B,meanCov,medianCov,std
0,chr7,35350,967,4.033958,2.313776,3.922525,12.226485,0.703237,0.822889,7.261974,...,3.480870,1.543389,3.009409,3.616486,6.796135,1.246768,0.697537,3.085826,2.008443,3.093574
1,chr7,35400,1017,5.966553,2.006082,6.716224,16.196813,0.598537,0.954275,11.765672,...,3.123285,1.334675,6.436252,7.599787,12.125656,1.186923,0.635850,4.657405,2.006082,5.103127
2,chr7,35450,1067,6.240286,0.937251,7.195536,15.730204,0.253026,0.658657,13.479243,...,1.620309,0.637129,7.829317,8.550406,12.945965,0.937569,0.412828,4.844926,1.126427,5.745554
3,chr7,36200,1112,10.687931,0.099191,0.089993,0.106274,0.000000,9.805522,0.418307,...,4.386008,6.327347,0.000000,0.000000,0.000000,0.000000,0.000000,3.020992,0.223438,3.935328
4,chr7,36250,1162,11.035757,0.562756,0.187812,0.272327,0.000000,8.633423,0.630645,...,4.821815,5.410101,0.000000,0.000000,0.000000,0.000000,0.000000,2.711400,0.562756,3.431450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47121,chr7,159144650,2423334,97.903753,123.464123,118.808504,97.370181,58.180198,29.482259,77.985534,...,67.930045,56.089291,57.901994,40.029333,34.920299,33.687667,39.555581,61.920583,58.180198,24.678553
47122,chr7,159144700,2423384,79.199359,96.427563,93.923455,80.314873,47.497280,27.042979,61.554787,...,50.492171,45.082345,51.378373,34.428946,29.734953,27.413931,30.876699,49.407998,47.497280,19.564811
47123,chr7,159144750,2423434,62.985317,77.984159,76.333716,67.963848,37.822975,23.901824,50.477091,...,39.814896,40.798204,49.407696,27.738447,26.165367,23.713525,25.671268,41.184198,39.315711,15.846882
47124,chr7,159144800,2423484,47.057357,60.899062,62.846488,52.525897,29.954749,18.653311,41.645936,...,32.562618,32.114580,44.670304,20.055998,22.178168,18.058186,17.837024,32.596834,30.363157,13.023941


### filter the coverage

In [27]:
mean_df['std'].max()

464.98562510909875

In [70]:
def filter_coverage(df, mincov=20, max_mean_std=20):
    filter_df = df.query('meanCov > @mincov and std < @max_mean_std')
    return filter_df

In [71]:
filter_df = filter_coverage(mean_df, mincov=50, max_mean_std=40)
filter_df.to_csv(f"{output_path}/PON_coverage_filter.csv", sep='\t', index=False)
tidy_df(filter_df).to_csv(f"{output_path}/PON_coverage_filter.tidy.csv", sep='\t', index=False)

### remove outliers in order to reduce noise

In [72]:
def remove_outliers(df, std_factor=2.5):
    # cycle through all sample cols and remove outliers
    for col in list(df.columns)[3:-3]:
        df.loc[np.abs(df['meanCov'] - df[col]) / df['std'] > std_factor, col] = np.nan
    return add_mean(df.iloc[:,:-3])

In [73]:
removed_df = remove_outliers(filter_df, std_factor=2)
removed_df.to_csv(f"{output_path}/PON_coverage_removed.csv", sep='\t', index=False)
tidy_df(removed_df).to_csv(f"{output_path}/PON_coverage_removed.tidy.csv", sep='\t', index=False)
removed_df

/Users/martinscience/miniconda/envs/HDR2-env/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/Users/martinscience/miniconda/envs/HDR2-env/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/Users/martinscience/miniconda/envs/HDR2-env/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

,Chr,Pos,ExonPos,001_B,002_B,003_B,004_B,005_B,006_B,008_B,...,034_B,036_B,039_B,040_B,041_B,042_B,044_B,meanCov,medianCov,std
0,chr7,208900,2041,67.696404,62.101496,45.964956,67.832666,52.917265,61.313901,64.000926,...,62.197505,66.947941,65.998275,44.059131,NaN,56.917446,59.470981,62.506986,62.506986,8.098135
1,chr7,208950,2091,59.729337,57.864635,37.591685,64.700906,53.714732,58.948958,63.631457,...,62.236616,66.772181,63.619871,NaN,NaN,53.017556,55.584702,59.648168,59.798555,7.916760
2,chr7,246400,2202,83.157166,98.773727,NaN,107.911228,112.908778,94.072158,152.089522,...,132.312173,119.478100,122.851830,114.177627,100.803529,92.081286,118.624036,112.937211,112.937211,16.937343
3,chr7,246450,2252,92.422921,112.771767,NaN,124.162839,129.816127,114.537209,170.760440,...,139.447119,130.474061,138.102737,136.486178,103.672124,108.389010,134.577235,127.762320,129.816127,19.300760
4,chr7,246500,2302,71.728304,89.627937,NaN,100.687921,96.976886,87.101801,131.726267,...,98.682387,94.767394,99.150309,110.189159,66.668738,80.256940,93.830621,96.456188,96.716537,17.548990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36177,chr7,159144400,2423084,176.032879,NaN,NaN,156.078233,81.798836,68.263513,104.009732,...,122.869685,104.285877,109.081358,92.954021,74.116148,81.763036,82.280923,105.321073,103.762820,24.162368
36178,chr7,159144450,2423134,198.446044,NaN,NaN,174.036869,88.291998,69.382020,121.308944,...,146.755270,121.960704,119.361887,98.523409,71.749681,86.884758,93.517441,115.555763,115.208127,29.841752
36179,chr7,159144550,2423234,163.614050,NaN,NaN,162.766849,78.144796,43.775639,124.670261,...,124.450882,98.496794,100.004593,70.877957,61.359102,63.580173,80.738748,98.133395,98.133395,28.730494
36180,chr7,159144600,2423284,127.913521,NaN,NaN,127.978738,67.966184,34.400578,101.981900,...,95.497642,73.780596,74.196486,50.992451,47.577913,46.459557,58.213516,75.584978,74.196486,23.525240


In [99]:
def make_PON_coverage(chrom, sample_list, config={
    'sample_PON_path': '.', # the folder with the sample/chrom coverages
    'outpath':'.',       # where the coverage files are saved
    'normCov':100,       # to what value are coverages normalized#
    'minCov': 20,        # only exonPositions with the average coverage above minCov are kept
    'max_mean_std': 20,  # only exonPositions with a coverage std below max_mean_std are kept
    'std_factor': 3,     # only exonPositions straighing within std_factor * std around meanCoverage are kept
}):
    
    # load all sample coverages for one chromosome
    cov_df = combine_coverage(config['sample_PON_path'], chrom, sample_list)
    
    # normalize and add mean values and std
    mean_df = add_mean(normalize_coverage(cov_df, norm_cov=config['normCov']))
    
    # filter hard regions and outlying data points
    filter_df = filter_coverage(mean_df, mincov=config['minCov'], max_mean_std=config['max_mean_std'])
    final_df = remove_outliers(filter_df, std_factor=config['std_factor'])
    
    # output
    file_name = f"{output_path}/PON_coverage.{chrom}.removed.csv"
    print(f"Saving filtered PON file {file_name} to {output_path}")
    final_df.to_csv(file_name, sep='\t', index=False)
    return final_df

In [105]:
config = {
    'sample_PON_path': bedCov_path,
    'output_path': output_path,
    'normCov':100,  
    'minCov': 20,   
    'max_mean_std': 20,
    'std_factor': 2,    
} 
sample_list = [f"{str(s).zfill(3)}_B" for s in range(45)]

df = make_PON_coverage("chr7", sample_list, config)

Reading 001_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/001_B.chr7.bedCov.
Reading 002_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/002_B.chr7.bedCov.
Reading 003_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/003_B.chr7.bedCov.
Reading 004_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/004_B.chr7.bedCov.
Reading 005_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/005_B.chr7.bedCov.
Reading 006_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/006_B.chr7.bedCov.
Reading 008_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/008_B.chr7.bedCov.
Reading 009_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/009_B.chr7.bedCov.
Reading 010_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/010_B.chr7.bedCov.
Reading 011_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/011_B.chr7.bedCov.


/Users/martinscience/miniconda/envs/HDR2-env/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/Users/martinscience/miniconda/envs/HDR2-env/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/Users/martinscience/miniconda/envs/HDR2-env/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

Saving filtered PON file /Users/martinscience/Dropbox/Icke/Work/somVar/tooldata/myCNVdata//output/PONcoverage/PON_coverage.chr7.removed.csv to /Users/martinscience/Dropbox/Icke/Work/somVar/tooldata/myCNVdata//output/PONcoverage


In [104]:
df

,Chr,Pos,ExonPos,001_B,002_B,003_B,004_B,005_B,006_B,008_B,...,034_B,036_B,039_B,040_B,041_B,042_B,044_B,meanCov,medianCov,std
0,chr7,208900,2041,67.696404,62.101496,45.964956,67.832666,52.917265,61.313901,64.000926,...,62.197505,66.947941,65.998275,44.059131,38.271137,56.917446,59.470981,61.537552,62.149501,9.206471
1,chr7,208950,2091,59.729337,57.864635,37.591685,64.700906,53.714732,58.948958,63.631457,...,62.236616,66.772181,63.619871,36.097696,35.840039,53.017556,55.584702,58.693629,59.729337,10.734254
2,chr7,209000,2141,37.046555,41.344317,21.895713,43.532465,35.804004,36.561527,40.781718,...,40.949112,44.406768,39.685364,19.818343,25.916788,30.785192,31.094976,36.962805,37.316476,7.557877
3,chr7,246500,2302,71.728304,89.627937,49.024721,100.687921,96.976886,87.101801,131.726267,...,98.682387,94.767394,99.150309,110.189159,66.668738,80.256940,93.830621,94.631901,94.767394,19.408199
4,chr7,248300,2339,124.953912,113.860840,68.036743,109.628216,91.865767,78.395770,108.135468,...,127.395374,132.055898,115.391409,124.133296,102.225398,99.746415,118.306111,110.864497,110.864497,18.346218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29881,chr7,159144250,2422935,67.270369,NaN,76.842372,62.462511,32.033049,27.355885,41.168174,...,39.015916,50.959307,52.819977,42.669367,27.497747,24.825641,36.385820,42.409291,40.179004,12.173417
29882,chr7,159144700,2423384,79.199359,96.427563,93.923455,80.314873,47.497280,27.042979,61.554787,...,50.492171,45.082345,51.378373,34.428946,29.734953,27.413931,30.876699,49.407998,47.497280,19.564811
29883,chr7,159144750,2423434,62.985317,77.984159,76.333716,67.963848,37.822975,23.901824,50.477091,...,39.814896,40.798204,49.407696,27.738447,26.165367,23.713525,25.671268,41.184198,39.315711,15.846882
29884,chr7,159144800,2423484,47.057357,60.899062,62.846488,52.525897,29.954749,18.653311,41.645936,...,32.562618,32.114580,44.670304,20.055998,22.178168,18.058186,17.837024,32.596834,30.363157,13.023941
